In [1]:
!pip install music21 tensorflow

In [ ]:
import numpy as np
import music21 as m21
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# MIDI 파일에서 음표 추출
def get_notes_from_midi(file):
    midi = m21.converter.parse(file)
    notes = []
    for element in midi.flat.notes:
        if isinstance(element, m21.note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, m21.chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

# 데이터 준비
notes = get_notes_from_midi('your_midi_file.mid')
unique_notes = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# 시퀀스 생성
sequence_length = 100
input_sequences = []
output_notes = []

for i in range(len(notes) - sequence_length):
    input_seq = notes[i:i + sequence_length]
    output_seq = notes[i + sequence_length]
    input_sequences.append([note_to_int[note] for note in input_seq])
    output_notes.append(note_to_int[output_seq])

# 데이터 변환
X = np.reshape(input_sequences, (len(input_sequences), sequence_length, 1))
X = X / float(len(unique_notes))  # 정규화

y = to_categorical(output_notes, num_classes=len(unique_notes))

# 모델 정의
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(len(unique_notes), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam')

# 모델 훈련
model.fit(X, y, epochs=100, batch_size=64)

# 음악 생성 함수
def generate_music(model, start_note, num_notes=100):
    pattern = [note_to_int[note] for note in start_note]
    generated_notes = []

    for _ in range(num_notes):
        input_pattern = np.reshape(pattern, (1, len(pattern), 1))
        input_pattern = input_pattern / float(len(unique_notes))
        prediction = model.predict(input_pattern, verbose=0)
        index = np.argmax(prediction)
        result = unique_notes[index]
        generated_notes.append(result)

        pattern.append(index)
        pattern = pattern[1:]

    return generated_notes

# 음악 생성
start_sequence = notes[:sequence_length]
generated_music = generate_music(model, start_sequence)

# 생성된 음악을 MIDI로 저장
def create_midi(generated_notes):
    offset = 0
    output_notes = []

    for note in generated_notes:
        if '.' in note or note.isdigit():
            notes_in_chord = note.split('.')
            chord = m21.chord.Chord([int(n) for n in notes_in_chord])
            chord.offset = offset
            output_notes.append(chord)
        else:
            new_note = m21.note.Note(note)
            new_note.offset = offset
            output_notes.append(new_note)
        offset += 0.5  # 각 음표 간 간격 설정

    midi_stream = m21.stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_music.mid')

# MIDI 파일 생성
create_midi(generated_music)
